In [1]:
from datetime import date, timedelta
from gamescout_db import db, cur 
from sklearn import linear_model
import pandas as pd
import requests
import json
import re

In [2]:
def computeGamma(hits, atbats):
    return hits - (.1 * atbats)

In [3]:
compiled = pd.read_sql('SELECT * FROM Compiled;', con=db)

games_played_b = compiled.groupby('P_ID').size()
compiled = compiled.join(pd.DataFrame(games_played_b, columns=['GAMES_PLAYED_B']), on=['P_ID'])

games_played_p = compiled.groupby(['starting_P_ID', 'G_ID']).size().reset_index().groupby('starting_P_ID').size()
compiled = compiled.join(pd.DataFrame(games_played_p, columns=['GAMES_PLAYED_P']), on=['starting_P_ID'])

compiled['P_ID'] = [777 if compiled['GAMES_PLAYED_B'][x] < 100 else compiled['P_ID'][x] for x in range(len(compiled))]
compiled['starting_P_ID'] = [888 if compiled['GAMES_PLAYED_P'][x] < 20 else compiled['starting_P_ID'][x] for x in range(len(compiled))]

In [ ]:
#compiled[compiled['starting_P_ID'] == 888]

In [4]:
def createModel():
    cur.execute("""
      SELECT
         COUNT(DISTINCT starting_P_ID)
      FROM
         Compiled
    """)
    num_pitchers = cur.fetchall()[0][0]

    cur.execute("""
      SELECT
         COUNT(DISTINCT P_ID)
      FROM
         Compiled
    """)
    num_batters = cur.fetchall()[0][0]

    data = compiled.copy()
    data = data[pd.notnull(data['GS1AGO'])]
    data = data[pd.notnull(data['GS2AGO'])]
    data = data[pd.notnull(data['GS3AGO'])]

    labels = data['GOT_HIT']
    data = data[['1_AGO', '2_AGO', '3_AGO', '4_AGO', '5_AGO', '6_AGO', '7_AGO',
                'GS1AGO', 'GS2AGO', 'GS3AGO', 'starting_P_ID', 'P_ID', 
                'hist_AB', 'hist_H']]

    pitch_dummies = pd.get_dummies(data['starting_P_ID']).iloc[:,1:num_pitchers] 
    bat_dummies = pd.get_dummies(data['P_ID']).iloc[:,1:num_batters]

    data['Gamma'] = data['hist_H'] - (.1 * data['hist_AB'])

    data = pd.concat([data, pitch_dummies], axis=1)
    data = pd.concat([data, bat_dummies], axis=1)

    data = data.drop('starting_P_ID', 1)
    data = data.drop('P_ID', 1)
    data = data.drop('hist_AB', 1)
    data = data.drop('hist_H', 1)

    y = labels
    x = data

    model = linear_model.LogisticRegression(class_weight='balanced')
    model.fit(x, y)

    return (model, pitch_dummies.columns, bat_dummies.columns)

In [5]:
def recentBatterGame(batter_id):
    cur.execute("""
      SELECT
         GOT_HIT, 1_AGO, 2_AGO, 3_AGO, 4_AGO, 
         5_AGO, 6_AGO, 7_AGO, hist_H, hist_AB
      FROM
         Compiled
      WHERE
         P_ID = %s
      ORDER BY G_DATE DESC
      LIMIT 1
    """, [batter_id])

    return cur.fetchall()[0]

In [6]:
def recentPitcherGame(pitcher_id):
    cur.execute("""
      SELECT
         GAME_SCORE_1AGO, GAME_SCORE_2AGO, GAME_SCORE_3AGO
      FROM
         PitcherStats
      WHERE
         P_ID = %s
      ORDER BY G_DATE DESC
      LIMIT 1
    """, [pitcher_id])
    try:
        return cur.fetchall()[0]
    except:
        print("Recent Pitcher Error: " + str(pitcher_id))

In [7]:
model, pitch_dummies, bat_dummies = createModel()

In [8]:
matchups = []
players = []
model_input = pd.DataFrame()

cur_day = date.today()
cur_season = cur_day.year
tomorrow = date.today() + timedelta(days=1)
url = "http://gd.mlb.com/components/game/mlb/year_{y}/month_{m:02d}/\
day_{d:02d}/".format(y=tomorrow.year, m=tomorrow.month, d=tomorrow.day)

games_index = requests.get(url).text
games = re.findall(r'> (gid.*mlb.*mlb.*)/</a>', games_index)


for game_id in games:
    info_url = '{url}{gid}/linescore.json'.format(url=url, gid=game_id)
    response = requests.get(info_url)
    info = json.loads(response.text)

    game = info['data']['game']
    matchups.append({
     'pitcher':  game['home_probable_pitcher']['id'],
     'opp_team': game['away_team_name']
     })

    matchups.append({
     'pitcher':  game['away_probable_pitcher']['id'],
     'opp_team': game['home_team_name']
     })

for matchup in matchups:
    cur.execute("""
     SELECT
        DISTINCT P_ID, NAME 
     FROM 
        BatterStats 
     WHERE 
        TEAM=%s AND YEAR(G_DATE)=%s AND 
        G_DATE >= DATE_ADD(%s,INTERVAL -5 DAY)
    """, [matchup['opp_team'], cur_season, cur_day]
    )
    rows = cur.fetchall()

    for row in rows:
        batter_id = row[0]
        batter_name = row[1]
        pitcher_id = matchup['pitcher']

        p_dummies = dict.fromkeys(pitch_dummies, [0])
        b_dummies = dict.fromkeys(bat_dummies, [0])

        if pitcher_id in p_dummies:
            p_dummies[pitcher_id] = [1]
        p_dummies_df = pd.DataFrame(p_dummies)

        if batter_id in b_dummies:
            b_dummies[batter_id] = [1]
        b_dummies_df = pd.DataFrame(b_dummies)

        batter = recentBatterGame(batter_id)
        pitcher = recentPitcherGame(pitcher_id)

        stats = pd.DataFrame({
            '1_AGO': batter[0],
            '2_AGO': batter[1],
            '3_AGO': batter[2],
            '4_AGO': batter[3],
            '5_AGO': batter[4],
            '6_AGO': batter[5],
            '7_AGO': batter[6],
            'GS1AGO': pitcher[0],
            'GS2AGO': pitcher[1],
            'GS3AGO': pitcher[2],
            'Gamma': [computeGamma(batter[8], batter[9])]
         })

        players.append({'batter_id': batter_id, 'batter_name': batter_name,'pitcher_id': pitcher_id})
        player_data = pd.concat([stats, p_dummies_df, b_dummies_df], axis=1)
        model_input = model_input.append(player_data)

In [ ]:
model_input

In [9]:
pred = model.predict_proba(model_input)[:,1]

In [10]:
players_df = pd.DataFrame(players)
results = pd.concat([players_df, pd.DataFrame(pred, columns=['Prediction'])], axis=1) 

In [11]:
results = results.sort_values(['Prediction'], ascending=False)

In [16]:
results[results['batter_name'] == 'Buster Posey']

,batter_id,batter_name,pitcher_id,Prediction
579,457763,Buster Posey,595191,0.804589


In [13]:
results.head()

,batter_id,batter_name,pitcher_id,Prediction
100,605141,Mookie Betts,594742,0.859484
309,514888,Jose Altuve,506433,0.840439
446,519184,Ben Revere,502043,0.836850
128,592518,Manny Machado,519144,0.833356
150,430945,Adam Jones,519144,0.828195


In [17]:
results[:50]

,batter_id,batter_name,pitcher_id,Prediction
100,605141,Mookie Betts,594742,0.859484
309,514888,Jose Altuve,506433,0.840439
446,519184,Ben Revere,502043,0.836850
128,592518,Manny Machado,519144,0.833356
150,430945,Adam Jones,519144,0.828195
567,452655,Denard Span,595191,0.824390
704,429664,Robinson Cano,543606,0.821699
181,408234,Miguel Cabrera,502706,0.819923
159,641313,Tim Anderson,605242,0.817816
101,456030,Dustin Pedroia,594742,0.816893
